In [1]:
import math

#Was man nicht alles online so findet...
def to_precision(x,p):
    """
    returns a string representation of x formatted with a precision of p

    Based on the webkit javascript implementation taken from here:
    https://code.google.com/p/webkit-mirror/source/browse/JavaScriptCore/kjs/number_object.cpp
    """

    x = float(x)

    if x == 0.:
        return "0." + "0"*(p-1)

    out = []

    if x < 0:
        out.append("-")
        x = -x

    e = int(math.log10(x))
    tens = math.pow(10, e - p + 1)
    n = math.floor(x/tens)

    if n < math.pow(10, p - 1):
        e = e -1
        tens = math.pow(10, e - p+1)
        n = math.floor(x / tens)

    if abs((n + 1.) * tens - x) <= abs(n * tens -x):
        n = n + 1

    if n >= math.pow(10,p):
        n = n / 10.
        e = e + 1

    m = "%.*g" % (p, n)

    if e < -2 or e >= p:
        out.append(m[0])
        if p > 1:
            out.append(".")
            out.extend(m[1:p])
        out.append('e')
        if e > 0:
            out.append("+")
        out.append(str(e))
    elif e == (p -1):
        out.append(m)
    elif e >= 0:
        out.append(m[:e+1])
        if e+1 < len(m):
            out.append(".")
            out.extend(m[e+1:])
    else:
        out.append("0.")
        out.extend(["0"]*-(e+1))
        out.append(m)

    return "".join(out)

#a: array das gerundet werden soll, p: Anzahl der signifikanten Stellen
def runden (a, p):
    ar = []
    i = 0
    while i<(len(a)):
        r=to_precision(a[i],p)
        ar.append(r)
        i+=1
    return(ar)

In [7]:
import numpy as np
def table(align,head,val,err,cap,lab):
    #align: entweder 'l','c','r'
    #align="c"
    
    #head: np.array an titeln der spalten['a','b','c']
    #head=np.array(['a','b','c'])
    
    #val: np.array mit [] mit den Values zu den titeln[[1,2,3],[2,4,6],[3,6,9]]
    #val=np.array([[1,3,3],[9,4,6],[3,3,4]])
    
    #err: np.array mit [] zu den Values,[0] wenn keine error existieren
    #,wenn das array 1 wert besitz wird dieser für alle genommen ([[0.1,0.2,0.1],[0],[0.1]],dtype=object)
    #err=np.array([[0.1,0.2,0.1],[0],[0.1]],dtype=object)

    #cap: die caption
    #cap="jo"
    
    #lab: label mit tab: davor
    #lab="1a" => label: Tab:1a
    
    size=len(val)
    fehler=np.zeros_like(val,dtype=np.dtype(np.float32))
    val=np.transpose(val)
    a=(align+"|")*(size-1)+align
    tex="\\begin{longtable}{"+a+"} \n \n\\caption{"+cap+"}\\\ \n \n"
    
    for string in head:
        tex+=string+"\t &"
        
    tex=tex[:-1]+"\\\\"+"\\hline\\hline"+"\n"+"\\endfirsthead \n \n\\caption{Fortsetzung "+cap+"}\\\ \n \n"
    
    for string in head:
        tex+=string+"\t &"
    tex=tex[:-1]+"\\\\"+"\\hline\hline\n\endhead"+"\n"+"\hline\n\multicolumn{4}{r}{Weiter auf der n{\"a}chsten Seite}\\\n\endfoot\hline\n\multicolumn{4}{r}{Ende} \\\n\endlastfoot\\\ \n \n"
    
    for i,array in enumerate(err): 
        if(len(array)==len(val)):
            fehler[i]=array
        elif(len(array)==1):
            fehler[i]=np.full(len(val),array)
        else:
            print("Error größe stimmt nicht mit values überein")
            print(i)
            print(array)

    fehler=np.transpose(fehler)
      
    for i,array in enumerate(val):
        print(array)       
        for j,v in enumerate(array):
            if(fehler[i,j]==0):
                tex=tex+str(v)+"\t&"
            else:
                tex=tex+str(v)+"$\\pm$"+str(fehler[i,j])+"\t&"
            
        tex=tex[:-1]+"\\\\ \n"
            
    tex=tex[:-4]+"\n"
        
    tex=tex+"\\label{Tab:"+lab+"} \n \n\\end{longtable}"
    return tex

In [3]:
import numpy as np
U=np.array([496,494,493,493,493,492,491,491,495,497,497,497,497,497,497,496,496,496,496,496,496,497,497,497,496,497,497
           ,497,497,497,498,498,498,498,498,498,498,499,499,499,499,499,499,498,498,498,498,498,498,498,498,499,499,499
           ,499,499,499,499,499,499])

U_fehler=1
U_err=np.full(len(U),U_fehler)
#Zeit:
t2=np.array([3.59,2.68,5.74,4.17,2.38,3.57,3.53,7.72,3.12,2.09,1.49,2.03,2.03,1.2,1.28,2.12,
            1.90,7.20,7.62,2.77,1.84,2.12,2.08,1.89,3.59,3.89,4.14,5.39,41.69,3.92,2.93,1.83,6.64,1.63,2.29,9.37,3.05,7.76,6.92,4.03,7.44,
            3.23,4.21,2.11,9.00,6.51,2.86,3.24,2.26,8.73,7.32,3.44,4.38,8.75,5.60,4.03,3.78,2.35,4.75,3.02])
t1=np.array([22.16,22.05,5.24,32.58,24.69,32.95,33.94,14.92,34.78,83.15,27.44,54.09,50.33,7.29,25.89,36.08,
            9.33,28.60,27.48,46.90,17.89,26.28,26.36,21.60,9.70,14.43,14.80,22.48,13.70,28.81,16.22,21.24,28.76,17.86,59.12,23.59,11.14,13.41,10.20,25.01,3.30,
            8.08,30.05,9.08,25.23,39.53,16.67,16.89,29.22,26.89,31.92,27.61,11.26,16.86,3.95,4.58,5.23,26.39,55.28,14.13])


t_fehler=0.25
t_err=np.full(len(t1),t_fehler)
t1_err=t2_err=t_err

#Strecke; x ist gemessen durchs Mikroskop; s ist umgerechnet in den Realen wert
x=np.full(len(t1),2)

x_fehler=0.2
x_err=np.full(len(x),x_fehler)

#Position
z=1.875 #zoom

z_err=1.875*0.01
s=x/z*10**(-3)
s_err=np.sqrt((1/z*10**(-4)*x_err)**2+
              (x*10**(-4)*z_err/z**2)**2)

#Geschwindigkeit
v1=s/t1
v1_err=np.sqrt((s_err/t1)**2+(s*t1_err/t1**2)**2)
v2=s/t2
v2_err=np.sqrt((s_err/t2)**2+(s*t2_err/t2**2)**2)


eta=1.81*10**(-5)
d=6*10**(-3)
poel=875.3
pl=1.29
p=874
p_err=0
g=9.81

Q=(v1+v2)*np.sqrt(v1)/U*eta**(3/2)*18*np.pi*d/np.sqrt(2*p*g)
Q=Q*10**19

#Als BSP
#Q=np.array([1,2,2,2,3,3,3,3,3,4,6,6,6,6,6,8,9,9,9,9,10]);


Q_err=9*np.pi*d*eta**(3/2)/np.sqrt(g*p)*np.sqrt(((3*v1+v2)/(U*np.sqrt(2*v1))*v1_err)**2+
                                                (np.sqrt(2*v1)/U*v2_err)**2+
                                                (np.sqrt(2*v1)*(v1+v2)/U**2*U_err)**2)

#welches eta? 
r=np.sqrt(9*eta*v1/(2*p*g))
r=r*10**(7)
#r_err überprüfen
r_err=1/(2*np.sqrt(9*eta*v1/(2*p*g)))*9*eta/(2*p*g)*v1_err
r_err=r_err*10**(7)

In [8]:
#gerundet
q_r=runden(Q, 3)
q_err_r=runden(Q_err,3)
U_r=runden(U,3)
U_err_r=runden(U_err,1)
t1_r=runden(t1,3)
t2_r=runden(t2,3)
r_r=runden(r,3)
r_err_r=runden(t2,3)
#Tabelle
head=np.array(['$U$ [V]','$t_1$ [s]','$t_2$ [s]','$Q$ [$10^{-19}\SI{}{\\ampere\s}$]','$r$ [10^{-7}m]'])
val=np.array([U_r, t1_r, t2_r, q_r,r_r])
err=np.array([U_err_r, t1_err, t2_err, q_err_r,r_err_r], dtype=object)
#x anpassen
x=table(align="c",head=head,val=val,err=err,cap="Messwerte und $Q$ für $x=\SI{3.0\pm0.1}{\mm}$",lab="Mess")
print(x)

['496' '22.2' '3.59' '9.64' '6.76']
['494' '22.1' '2.68' '12.5' '6.78']
['493' '5.24' '5.74' '22.5' '13.9']
['493' '32.6' '4.17' '6.68' '5.58']
['493' '24.7' '2.38' '13.1' '6.41']
['492' '33.0' '3.57' '7.64' '5.55']
['491' '33.9' '3.53' '7.60' '5.46']
['491' '14.9' '7.72' '7.20' '8.24']
['495' '34.8' '3.12' '8.32' '5.40']
['497' '83.2' '2.09' '7.52' '3.49']
['497' '27.4' '1.49' '18.9' '6.08']
['497' '54.1' '2.03' '9.72' '4.33']
['497' '50.3' '2.03' '10.1' '4.49']
['497' '7.29' '1.20' '50.3' '11.8']
['497' '25.9' '1.28' '22.5' '6.26']
['496' '36.1' '2.12' '11.7' '5.30']
['496' '9.33' '1.90' '29.1' '10.4']
['496' '28.6' '7.20' '4.56' '5.95']
['496' '27.5' '7.62' '4.48' '6.07']
['496' '46.9' '2.77' '7.82' '4.65']
['496' '17.9' '1.84' '19.9' '7.53']
['497' '26.3' '2.12' '13.9' '6.21']
['497' '26.4' '2.08' '14.1' '6.20']
['497' '21.6' '1.89' '17.3' '6.85']
['496' '9.70' '3.59' '17.2' '10.2']
['497' '14.4' '3.89' '12.0' '8.38']
['497' '14.8' '4.14' '11.2' '8.27']
['497' '22.5' '5.39' '6.78' 